텍스트 + 이미지 생성 모델
  - 1. 미리 훈련된 모델을 로드
    - DALL-E, DALL-E2는 openAI에서 제공하는 모델, 텍스트와 이미지를 결합해서 학습되었음
  - 2. 텍스트 프롬프트를 사용해서 이미지  생성
    - 사용자가 제공한 텍스트를 프롬프트 기반으로 모델은 텍스트의 의미를 이해하고 이를 시각적으로 표현하는 이미지를 생성

openai는 유료버전이므로 무료로 사용가능한 hugging face의 stable diffusion 모델을 사용 할 수 있음

In [1]:
!pip install diffusers transformers accelerate torch

In [2]:
# 허깅페이스 로그인
from huggingface_hub import notebook_login

notebook_login()

모델 다운로드 및 설정

In [2]:
import torch
from diffusers import StableDiffusionPipeline

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# stablediffusion 파이프 라인 로드
pipe = StableDiffusionPipeline.from_pretrained("sd-legacy/stable-diffusion-v1-5", torch_dtype=torch.float16)
pipe = pipe.to(device)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

이미지 생성 함수

In [6]:
def generate_image(prompt,output_path='generated_image.png'):
  '''
  텍스트 프롬프트 기반으로 이미지 생성(영어로)
  Args:
    prompt (str) : 텍스트 프롬프트
    output_path (str) : 생성된 이미지 저장 경로
  '''
  try:
    # 이미지 생성
    image = pipe(prompt).images[0]
    # 이미지 저장
    image.save(output_path)
  except Exception as e:
    print(f'이미지 생성 오류: {e}')


In [18]:
generate_image('A rabbit running on Mars')

  0%|          | 0/50 [00:00<?, ?it/s]

최적화
  - 스케일링을 변경
  -